# NLP Assignment

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/javierarenas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/javierarenas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/javierarenas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
import re

In [3]:
def unicodetoascii(text):
    TEXT = (text.
    		replace('â\x80\x99', "'").
            replace('â\xc3\xa9', 'e').
            replace('â\x80\x90', '-').
            replace('â\x80\x91', '-').
            replace('â\x80\x92', '-').
            replace('â\x80\x93', '-').
            replace('â\x80\x94', '-').
            replace('â\x80\x94', '-').
            replace('â\x80\x98', "'").
            replace('â\x80\x9b', "'").
            replace('â\x80\x9c', '"').
            replace('â\x80\x9c', '"').
            replace('â\x80\x9d', '"').
            replace('â\x80\x9e', '"').
            replace('â\x80\x9f', '"').
            replace('â\x80\xa6', '...').#
            replace('â\x80\xb2', "'").
            replace('â\x80\xb3', "'").
            replace('â\x80\xb4', "'").
            replace('â\x80\xb5', "'").
            replace('â\x80\xb6', "'").
            replace('â\x80\xb7', "'").
            replace('â\x81\xba', "+").
            replace('â\x81\xbb', "-").
            replace('â\x81\xbc', "=").
            replace('â\x81\xbd', "(").
            replace('â\x81\xbe', ")").
            replace('\'', '')
           )
    return TEXT

In [4]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()


In [5]:
from pymongo import MongoClient
client = MongoClient()
db = client.nyt_dump
coll = db.articles

documents = ['\n'.join(article['content']) for article in coll.find()]

articles = []

for i,document in enumerate(coll.find(projection={'content':1})):
    
    articles.append(document.get('content'))
    articles[i] = ''.join(articles[i])
    articles[i] = unicodetoascii(articles[i])
    articles[i] = re.sub('\n','',articles[i])
    articles[i] = re.sub('\s ',' ',articles[i])
    articles[i] = remove_accents(articles[i])


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


tokens = [word_tokenize(article) for article in articles]


In [7]:
tokens_lower = [[word.lower() for word in sent]
                 for sent in tokens]

In [8]:
import string

stopwords_ = set(stopwords.words('english'))

punctuation_ = set(string.punctuation)

special = set("',``,''".split(','))


def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_ and not w in special])

tokens_filtered = list(map(filter_tokens, tokens_lower))

In [9]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

tokens_porter = [[porter.stem(token) for token in art] for art in tokens_filtered]
tokens_snow = [[snowball.stem(token) for token in art] for art in tokens_filtered]
tokens_wordnet = [[wordnet.lemmatize(token) for token in art] for art in tokens_filtered]

        

In [10]:
print(tokens_porter[0][0:10])
print(tokens_snow[0][0:10])
print(tokens_wordnet[0][0:10])

['hey', 'man', 'phone', 'said', 'still', 'come', 'tonight', 'took', 'moment', 'realiz']
['hey', 'man', 'phone', 'said', 'still', 'come', 'tonight', 'took', 'moment', 'realiz']
['hey', 'man', 'phone', 'said', 'still', 'coming', 'tonight', 'took', 'moment', 'realize']


### I'll use tokens wordnet since it takes the context of the sentence and makes it more readable.

In [11]:
vocab = set()

for tokens in tokens_wordnet:
    for word in tokens:
        
        vocab.add(word)

In [12]:
len(vocab)

36118

In [13]:
vocab_dict = {}

for i, word in enumerate(vocab):

    vocab_dict.update({word:i})

In [14]:
import numpy as np

vocab_matrix = np.empty([len(tokens_wordnet), len(vocab_dict)])


In [15]:
vocab_matrix.shape

(999, 36118)

In [16]:
from collections import Counter

term_occ = list(map(lambda art : Counter(art), tokens_wordnet))

In [17]:
for i,terms in enumerate(term_occ):
    for j,vocab in enumerate(vocab_dict):
    
        vocab_matrix[i,j] = term_occ[i][vocab]

In [18]:
docc_freq = []

for i in range(vocab_matrix.shape[1]):
    
    res = len(np.where(vocab_matrix[:,i] > 0)[0])
    docc_freq.append(res/len(articles))


In [19]:
term_freq = np.empty([vocab_matrix.shape[0], vocab_matrix.shape[1]])

for i in range(vocab_matrix.shape[0]):
    
    term_freq[i,:] = vocab_matrix[i,:] / len(tokens_wordnet[i])

/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [20]:
tf_idf = term_freq * np.log(1/np.array(docc_freq).reshape(1,len(docc_freq)))

In [21]:
tf_idf.shape

(999, 36118)

In [22]:
for i in range(tf_idf.shape[0]):

    tf_idf[i,:] = tf_idf[i,:] / np.linalg.norm(tf_idf[i,:],2)

## Using Sklearn

In [36]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vect = CountVectorizer(stop_words='english')
word_counts = vect.fit_transform(articles)

In [24]:
word_counts = word_counts.toarray()

In [25]:
def tokenize(docs):
    
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.tokenize import word_tokenize
    '''
    INPUT: string
    OUTPUT: list of strings

    Tokenize and stem/lemmatize the document.
    '''
    
    tokens_wordnet = [wordnet.lemmatize(token) for token in word_tokenize(docs.lower())]
    
    return tokens_wordnet

In [26]:
vect = CountVectorizer(stop_words='english', tokenizer = tokenize)
word_counts = vect.fit_transform(articles)

/Library/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [27]:
words = vect.get_feature_names()

In [ ]:
21804

In [32]:
print("sklearn count of 'dinner':", word_counts[0, words.index('dinner')])
print("my count of 'dinner':", vocab_matrix[0, 21804])

sklearn count of 'dinner': 2
my count of 'dinner': 2.0


In [37]:
tfidfvect = TfidfVectorizer(stop_words='english', tokenizer=tokenize)
tfidf_vectorized = tfidfvect.fit_transform(articles)

/Library/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [40]:
words_tfidf = tfidfvect.get_feature_names()
print("sklearn tfidf of 'dinner':", tfidf_vectorized[0, words_tfidf.index('dinner')])
print("my tfidf of 'dinner':", tf_idf[0, 21804])

sklearn tfidf of 'dinner': 0.05596800166849005
my tfidf of 'dinner': 0.058522997606060646
